In [3]:
import cv2
import numpy as np 
import math
import glob
from matplotlib import pyplot as plt

In [18]:
money_template = []
money = []
files = glob.glob("dataset\\500000\\*.jpg")
print(files)
for item in files:
    img = cv2.imread(item, 0)
    money_template.append(img)

['dataset\\500000\\1633072080426.jpg', 'dataset\\500000\\1633072090718.jpg', 'dataset\\500000\\1633072122434.jpg', 'dataset\\500000\\1633072138062.jpg', 'dataset\\500000\\1633072154074.jpg', 'dataset\\500000\\1633072160646.jpg', 'dataset\\500000\\1633072180274.jpg', 'dataset\\500000\\1633072190597.jpg', 'dataset\\500000\\1633072210562.jpg', 'dataset\\500000\\1633072216861.jpg', 'dataset\\500000\\1633072221697.jpg', 'dataset\\500000\\1633072227501.jpg', 'dataset\\500000\\1633072236217.jpg', 'dataset\\500000\\1633072253549.jpg', 'dataset\\500000\\1633072267129.jpg', 'dataset\\500000\\1633072290309.jpg', 'dataset\\500000\\1633072309544.jpg', 'dataset\\500000\\1633072325340.jpg', 'dataset\\500000\\1633072329800.jpg', 'dataset\\500000\\1633073226960.jpg', 'dataset\\500000\\1633073240340.jpg', 'dataset\\500000\\1633073248416.jpg', 'dataset\\500000\\1633073267355.jpg', 'dataset\\500000\\1633073280623.jpg', 'dataset\\500000\\1633073289163.jpg', 'dataset\\500000\\1633074317991.jpg', 'dataset\\5

In [19]:
for i , im in enumerate(money_template):
    img_blur = cv2.blur(im, (5,5))
    img_canny = cv2.Canny(img_blur,30, 40)
    kernel = np.ones((30,30),np.uint8)
    img_closed = cv2.morphologyEx(img_canny, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(img_closed,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnt = None
    s_max_large = 0
    for item in contours:
        s = cv2.contourArea(item)
        if(s > s_max_large):
            s_max_large = s
            cnt = item
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    #xoay ảnh
    angle = rect[2]
    RM = cv2.getRotationMatrix2D((cx,cy),angle,1)
    # tính độ dài chiều dọc (dist1) và chiều ngang(dist2). 
    # nếu chiều dọc > chiều ngang đồng tiền nằm dọc khi đó cần trừ đi 90 độ xoay
    dist1 = int(math.sqrt((box[1][0]  - box[0][0])**2 + (box[1][1]  - box[0][1])**2))
    dist2 = int(math.sqrt((box[3][0]  - box[0][0])**2 + (box[3][1]  - box[0][1])**2))
    if(dist1 > dist2):
        angle -= 90
    rows, cols = img.shape
    # thực hiện xoay lại ảnh
    RM = cv2.getRotationMatrix2D((cx,cy),angle,1)
    img_rotated = cv2.warpAffine(im,RM,(cols,rows))
    img_blur2 = cv2.blur(img_rotated, (5,5))
    img_canny2 = cv2.Canny(img_blur2,30, 40)
    kernel2 = np.ones((30,30),np.uint8)
    img_closed2 = cv2.morphologyEx(img_canny2, cv2.MORPH_CLOSE, kernel2)
    contours2, _ = cv2.findContours(img_closed2,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnt2 = None
    s_max_large2 = 0
    for item in contours2:
        s = cv2.contourArea(item)
        if(s > s_max_large2):
            s_max_large2 = s
            cnt2 = item
    x,y,w,h = cv2.boundingRect(cnt2)
    img_money = img_rotated[y:y+h, x:x+w]
    money.append(img_money)
    i1=str(i+9)
    cv2.imwrite("template\\out1\\" + "500("+i1+").png", im)